In [ ]:
# generates inference slides and ground truth from the slides and annotations
# inference slides will be used by the trained model to generate predictions, which can then be compared with the ground truth

# note to self. can we use the same ground truth slide combiner for both inference and prediction slides? (check __debug_inf.ipynb)

from script3_inference import get_slices, join_pieces
from slide_utils import load_annotations, get_slide_offset, Bounds
import shutil
from settings import load_env
import os
from openslide import open_slide


os.environ['overrides'] = '__local.env'
load_env()


def generate_and_save_inference_slides(slide_name, region_name,  level, overlap):
    slidefile = os.environ.get(f'slidefile_{slide_name}')
    annotationsfile = os.environ[f"annotations_{slide_name}"]

    slide = open_slide(slidefile)
    print(f"Loaded slidefile {slidefile}")

    annotations = load_annotations(annotationsfile, get_slide_offset(slide))
    print(f"Loaded annotations: filename={annotationsfile}, outsides={len(annotations.outsides)}, insides={len(annotations.insides)}")

    region = annotations.regions[region_name]
    bounds = Bounds(
        topleft=region[0],
        topright=region[1],
        bottomleft=region[3],
        bottomright=region[2],
        zoom_level=level,
    )
    print(bounds)

    print(f"Getting slices from {slide_name}/{region_name}. size={bounds.get_size()}, level={level}, overlap={overlap}")
    pieces = get_slices(slide, bounds, level, overlap)
    print(f"got slices: {len(pieces[0])} x {len(pieces[1])}")


    # save pieces to disk

    target = f"{os.environ.get('inference_slides')}/{slide_name}/lvl{level}_overlap{int(overlap*100)}"
    print(f"Saving {len(pieces) * len(pieces[1])} pieces to {target}")

    if os.path.exists(target):
        print("Folder already exists, removing it")
        shutil.rmtree(target)

    os.makedirs(target)

    for row in range(len(pieces)):
        print(".", end="")
        for col in range(len(pieces[row])):
            filename = f"{target}/row{row}_col{col}_orig.png"
            pieces[row][col].save(filename)

    print(" Done!")

    original = join_pieces(pieces, overlap, True, bounds.get_size())
    original.thumbnail((1024, 1024))
    original.save(f"{os.environ.get('inference_slides')}/{slide_name}/lvl{level}_overlap{int(overlap*100)}_preview.png")


In [ ]:
# slide_name = 'beta'
# region_name = 'Test Region C'
#
# generate_and_save_inference_slides(slide_name, region_name, 1, 0.5)
# generate_and_save_inference_slides(slide_name, region_name, 1, 0.25)
# generate_and_save_inference_slides(slide_name, region_name, 2, 0.5)
# generate_and_save_inference_slides(slide_name, region_name, 2, 0.25)
#


slide_name = 'gamma'
region_name = 'Test Region Gamma2'

generate_and_save_inference_slides(slide_name, region_name, 1, 0.25)
generate_and_save_inference_slides(slide_name, region_name, 1, 0.5)
# generate_and_save_inference_slides(slide_name, region_name, 2, 0.5)
# generate_and_save_inference_slides(slide_name, region_name, 2, 0.25)

### Ground Truth

In [ ]:
import numpy as np
from slide_utils import get_localized_polygon, draw_polygons
from PIL import Image


def generate_ground_truth(slide_name: str, region_name: str, level: int):
    CH_BORDER = 0
    CH_TUBULE = 1
    CH_BACKGR = 2

    annotations = load_annotations(os.environ.get(f'annotations_{slide_name}'), (0,0))
    region = annotations.regions[region_name]

    bounds = Bounds(
        topleft=region[0],
        topright=region[1],
        bottomleft=region[3],
        bottomright=region[2],
        zoom_level=level,
    )
    img_seg = Image.new("RGB", bounds.get_size())

    localized_outsides = [get_localized_polygon(bounds, poly) for poly in annotations.outsides]
    localized_insides = [get_localized_polygon(bounds, poly) for poly in annotations.insides]

    draw_polygons(img_seg, localized_outsides, fill="red")
    draw_polygons(img_seg, localized_insides, fill="#00ff00")

    # save several versions of the generated image
    workdir = os.environ['inference_slides']
    save_dir = f'{workdir}/{slide_name}'

    # 1-channels postprocessing output.
    img_seg.save(f'{save_dir}/ground_truth_level{level}_postproc.png')

    # 2-channel borders-only version. maybe that's all we need?
    brdr = np.array(img_seg.copy())[:,:,CH_BORDER]
    borders = Image.fromarray(brdr)
    borders.save(f'{save_dir}/ground_truth_level{level}_borders.png')

    # image for raw model output. add blue channel where there are no border nor tubules
    blue_img = np.array(img_seg.copy())
    blue_img[:,:,CH_BACKGR] = (blue_img[:,:,CH_BORDER]==0) * (blue_img[:,:,CH_TUBULE]==0) * 255
    blue_img = Image.fromarray(blue_img)
    blue_img.save(f'{save_dir}/ground_truth_level{level}_model.png')

    return img_seg, borders, blue_img

In [ ]:
# generate_ground_truth('alpha', 'Test Region A', 1)
# generate_ground_truth('alpha', 'Test Region A', 2)

generate_ground_truth('beta', 'Test Region C', 1)
generate_ground_truth('beta', 'Test Region C', 2)

generate_ground_truth('gamma', 'Test Region Gamma2', 1)
generate_ground_truth('gamma', 'Test Region Gamma2', 2)